In [30]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
#print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

#### Importing dependencies

In [31]:
from xgboost import XGBRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import accuracy_score,mean_absolute_error

#### Reading the training dataset

In [32]:
train_data = pd.read_csv("train.csv")
train_data.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [33]:
#not using survived attr as it is the output 
#not using passengerId as it is would cause a bias and not related to the prediction
train_features = ['Pclass', 'Name', 'Sex', 'Age', 'SibSp',
       'Parch', 'Ticket', 'Fare', 'Cabin', 'Embarked']
train_data_X= train_data[train_features]

#Sex is necessary attribute and needs to be converted in binary categories
train_data_X=train_data_X.replace(to_replace="female", value=1)
train_data_X=train_data_X.replace(to_replace="male", value=0)

#fetching the numerical attributes
numerical_attr = list(train_data_X.dtypes[train_data_X.dtypes !='object'].index)
X = train_data_X[numerical_attr] #X is the required training data
y = train_data.Survived #y is set of predictions of training data

In [34]:
#spliting the data
train_X,test_X,train_y,test_y = train_test_split(X,y,test_size = 0.3,random_state=1)
#filling the missing Age values with mean values
train_X['Age'].fillna(X.Age.mean(),inplace=True)
test_X['Age'].fillna(X.Age.mean(),inplace=True)


C:\Users\Admin\Anaconda3\lib\site-packages\pandas\core\generic.py:5430: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._update_inplace(new_data)


In [35]:
#Using XGboost 
model = XGBRegressor(n_estimates = 1000,learning_rate=0.01)
model.fit(train_X,train_y)
y_pred1 = model.predict(test_X)
y_pred1 = np.round(y_pred1)

print("With XGBoost")
mae = mean_absolute_error(test_y,y_pred1)
print("Mean absolute error is : {}".format(mae))
a = accuracy_score(test_y,y_pred1)
print("Accuracy score is : {} ".format(a))

With XGBoost
Mean absolute error is : 0.21641791044776118
Accuracy score is : 0.7835820895522388 


In [36]:
#Using RandomForestRegressor
model2 = RandomForestRegressor(random_state=1)
model2.fit(train_X,train_y)
y_pred2 = model2.predict(test_X)
y_pred2 = np.round(y_pred2)

print("With RandomForestRegressor")
mae = mean_absolute_error(test_y,y_pred2)
print("Mean absolute error is : {}".format(mae))
a = accuracy_score(test_y,y_pred2)
print("Accuracy score is : {} ".format(a))

With RandomForestRegressor
Mean absolute error is : 0.23507462686567165
Accuracy score is : 0.7649253731343284 


C:\Users\Admin\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:248: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


In [37]:
#Using DecisionTreeRegressor 
model3 = DecisionTreeRegressor(random_state=1)
model3.fit(train_X,train_y)
y_pred3 = model3.predict(test_X)
y_pred3 = np.round(y_pred3)

print("With DecisionTreeeRegressor")
mae = mean_absolute_error(test_y,y_pred3)
print("Mean absolute error is : {}".format(mae))
a = accuracy_score(test_y,y_pred3)
print("Accuracy score is : {} ".format(a))


With DecisionTreeeRegressor
Mean absolute error is : 0.26119402985074625
Accuracy score is : 0.7388059701492538 


## Now working with the test.csv dataset 

In [38]:
my_test_data = pd.read_csv("test.csv")
my_test_data = my_test_data.replace(to_replace="female", value=1)
my_test_data = my_test_data.replace(to_replace="male", value=0)
my_test_X = my_test_data[numerical_attr]
my_test_X['Age'].fillna(my_test_X.Age.mean(),inplace=True)
my_test_X = my_test_X.fillna(my_test_X.mean()).copy()

C:\Users\Admin\Anaconda3\lib\site-packages\pandas\core\generic.py:5430: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._update_inplace(new_data)


In [39]:
predictions = model.predict(my_test_X)
predictions = np.round(predictions)
predictions = predictions.astype(int)

### sending predictions to csv file

In [40]:
submission = pd.DataFrame({
    "PassengerId" : my_test_data['PassengerId'],
    "Survived" : predictions
})
submission.to_csv('titanic_pred.csv',index=False)
submission.dtypes

PassengerId    int64
Survived       int32
dtype: object